In [ ]:
#default_exp tabular.core

In [ ]:
#export
from fastai2.torch_basics import *
from fastai2.data.all import *

In [ ]:
from nbdev.showdoc import *

In [ ]:
#export
pd.set_option('mode.chained_assignment','raise')

# Tabular core

> Basic function to preprocess tabular data before assembling it in a `DataBunch`.

## Tabular -

In [ ]:
#export
class _TabIloc:
    "Get/set rows by iloc and cols by name"
    def __init__(self,to): self.to = to
    def __getitem__(self, idxs):
        df = self.to.items
        if isinstance(idxs,tuple):
            rows,cols = idxs
            cols = df.columns.isin(cols) if is_listy(cols) else df.columns.get_loc(cols)
        else: rows,cols = idxs,slice(None)
        return self.to.new(df.iloc[rows, cols])

In [ ]:
#export
class Tabular(CollBase, GetAttr, FilteredBase):
    "A `DataFrame` wrapper that knows which cols are cont/cat/y, and returns rows in `__getitem__`"
    _default='items'
    def __init__(self, df, procs=None, cat_names=None, cont_names=None, y_names=None, block_y=CategoryBlock, splits=None, do_setup=True):
        if splits is None: splits=[range_of(df)]
        df = df.iloc[sum(splits, [])].copy()
        super().__init__(df)
        
        self.y_names = L(y_names)
        if block_y is not None: 
            if callable(block_y): block_y = block_y()
            procs = L(procs) + block_y.type_tfms
        self.cat_names,self.cont_names,self.procs = L(cat_names),L(cont_names),Pipeline(procs, as_item=True)
        self.split = len(splits[0])
        if do_setup: self.setup()

    def subset(self, i): return self.new(self.items[slice(0,self.split) if i==0 else slice(self.split,len(self))])
    def copy(self): self.items = self.items.copy(); return self
    def new(self, df): return type(self)(df, do_setup=False, block_y=None, **attrdict(self, 'procs','cat_names','cont_names','y_names'))
    def show(self, max_n=10, **kwargs): display_df(self.all_cols[:max_n])
    def setup(self): self.procs.setup(self)
    def process(self): self.procs(self)
    def iloc(self): return _TabIloc(self)
    def targ(self): return self.items[self.y_names]
    def all_col_names (self): return self.cat_names + self.cont_names + self.y_names
    def n_subsets(self): return 2

properties(Tabular,'iloc','targ','all_col_names','n_subsets')

In [ ]:
#export
class TabularPandas(Tabular):
    def transform(self, cols, f): self[cols] = self[cols].transform(f)

In [ ]:
#export
def _add_prop(cls, nm):
    @property
    def f(o): return o[list(getattr(o,nm+'_names'))]
    @f.setter
    def fset(o, v): o[getattr(o,nm+'_names')] = v
    setattr(cls, nm+'s', f)
    setattr(cls, nm+'s', fset)

_add_prop(Tabular, 'cat')
_add_prop(Tabular, 'cont')
_add_prop(Tabular, 'y')
_add_prop(Tabular, 'all_col')

In [ ]:
df = pd.DataFrame({'a':[0,1,2,0,2], 'b':[0,0,0,0,1]})
to = TabularPandas(df, cat_names='a')
t = pickle.loads(pickle.dumps(to))
test_eq(t.items,to.items)
test_eq(to.all_cols,to[['a']])
to.show() # only shows 'a' since that's the only col in `TabularPandas`

,a
0,0
1,1
2,2
3,0
4,2


In [ ]:
#export
class TabularProc(InplaceTransform):
    "Base class to write a non-lazy tabular processor for dataframes"
    def setup(self, items=None):
        super().setup(getattr(items,'train',items))
        # Procs are called as soon as data is available
        return self(items.items if isinstance(items,DataSource) else items)

In [ ]:
#export
def _apply_cats (voc, add, c): return c.cat.codes+add if is_categorical_dtype(c) else c.map(voc[c.name].o2i)
def _decode_cats(voc, c): return c.map(dict(enumerate(voc[c.name].items)))

In [ ]:
#export
class Categorify(TabularProc):
    "Transform the categorical variables to that type."
    order = 1
    def setups(self, to):
        self.classes = {n:CategoryMap(to.iloc[:,n].items, add_na=(n in to.cat_names)) for n in to.cat_names}
    def encodes(self, to): to.transform(to.cat_names, partial(_apply_cats, self.classes, 1))
    def decodes(self, to): to.transform(to.cat_names, partial(_decode_cats, self.classes))
    def __getitem__(self,k): return self.classes[k]

In [ ]:
#export
@Categorize
def setups(self, to:Tabular): 
    if len(to.y_names) > 0: self.vocab = CategoryMap(to.iloc[:,to.y_names[0]].items)
    return self(to)

@Categorize
def encodes(self, to:Tabular): 
    to.transform(to.y_names, partial(_apply_cats, {n: self.vocab for n in to.y_names}, 0))
    return to
  
@Categorize
def decodes(self, to:Tabular): 
    to.transform(to.y_names, partial(_decode_cats, {n: self.vocab for n in to.y_names}))
    return to

In [ ]:
show_doc(Categorify, title_level=3)

<h3 id="Categorify" class="doc_header"><code>class</code> <code>Categorify</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>Categorify</code>(**`enc`**=*`None`*, **`dec`**=*`None`*, **`split_idx`**=*`None`*, **`as_item`**=*`False`*, **`order`**=*`None`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Transform the categorical variables to that type.

In [ ]:
df = pd.DataFrame({'a':[0,1,2,0,2]})
to = TabularPandas(df, Categorify, 'a')
cat = to.procs.categorify
test_eq(cat['a'], ['#na#',0,1,2])
test_eq(to.a, [1,2,3,1,3])

In [ ]:
df1 = pd.DataFrame({'a':[1,0,3,-1,2]})
to1 = to.new(df1)
to1.process()
#Values that weren't in the training df are sent to 0 (na)
test_eq(to1.a, [2,1,0,0,3])
to2 = cat.decode(to1)
test_eq(to2.a, [1,0,'#na#','#na#',2])

In [ ]:
#test with splits
cat = Categorify()
df = pd.DataFrame({'a':[0,1,2,3,2]})
to = TabularPandas(df, cat, 'a', splits=[[0,1,2],[3,4]])
test_eq(cat['a'], ['#na#',0,1,2])
test_eq(to['a'], [1,2,3,0,3])

In [ ]:
df = pd.DataFrame({'a':pd.Categorical(['M','H','L','M'], categories=['H','M','L'], ordered=True)})
to = TabularPandas(df, Categorify, 'a')
cat = to.procs.categorify
test_eq(cat['a'], ['#na#','H','M','L'])
test_eq(to.a, [2,1,3,2])
to2 = cat.decode(to)
test_eq(to2.a, ['M','H','L','M'])

In [ ]:
#test with targets
cat = Categorify()
df = pd.DataFrame({'a':[0,1,2,3,2], 'b': ['a', 'b', 'a', 'b', 'b']})
to = TabularPandas(df, cat, 'a', splits=[[0,1,2],[3,4]], y_names='b')
test_eq(to.procs.vocab, ['a', 'b'])
test_eq(to.b, [0,1,0,1,1])
to2 = to.procs.decode(to)
test_eq(to2.b, ['a', 'b', 'a', 'b', 'b'])

In [ ]:
#export
class NormalizeTab(TabularProc):
    "Normalize the continuous variables."
    order = 2
    def setups(self, dsrc): self.means,self.stds = dsrc.conts.mean(),dsrc.conts.std(ddof=0)+1e-7
    def encodes(self, to): to.conts = (to.conts-self.means) / self.stds
    def decodes(self, to): to.conts = (to.conts*self.stds ) + self.means

In [ ]:
#export
@Normalize
def setups(self, to:Tabular): 
    self.means,self.stds = to.conts.mean(),to.conts.std(ddof=0)+1e-7
    return self(to)

@Normalize
def encodes(self, to:Tabular): 
    to.conts = (to.conts-self.means) / self.stds
    return to
  
@Normalize
def decodes(self, to:Tabular): 
    to.conts = (to.conts*self.stds ) + self.means
    return to

In [ ]:
show_doc(Normalize, title_level=3)

<h3 id="Normalize" class="doc_header"><code>class</code> <code>Normalize</code><a href="https://github.com/fastai/fastai2/tree/master/fastai2/data/transforms.py#L269" class="source_link" style="float:right">[source]</a></h3>

> <code>Normalize</code>(**`mean`**, **`std`**, **`dim`**=*`1`*, **`ndim`**=*`4`*, **`cuda`**=*`True`*) :: `Transform`

Normalize/denorm batch of [`TensorImage`](/torch.core.html#TensorImage)

In [ ]:
norm = Normalize()
df = pd.DataFrame({'a':[0,1,2,3,4]})
to = TabularPandas(df, norm, cont_names='a')
x = np.array([0,1,2,3,4])
m,s = x.mean(),x.std()
test_eq(norm.means['a'], m)
test_close(norm.stds['a'], s)
test_close(to.a.values, (x-m)/s)

TypeError: __init__() missing 2 required positional arguments: 'mean' and 'std'

In [ ]:
df1 = pd.DataFrame({'a':[5,6,7]})
to1 = to.new(df1)
to1.process()
test_close(to1['a'].values, (np.array([5,6,7])-m)/s)
to2 = norm.decode(to1)
test_close(to2.a.values, [5,6,7])

In [ ]:
norm = Normalize()
df = pd.DataFrame({'a':[0,1,2,3,4]})
to = TabularPandas(df, norm, cont_names='a', splits=[[0,1,2],[3,4]])
x = np.array([0,1,2])
m,s = x.mean(),x.std()
test_eq(norm.means['a'], m)
test_close(norm.stds['a'], s)
test_close(to['a'].values, (np.array([0,1,2,3,4])-m)/s)

In [ ]:
#export
class FillStrategy:
    "Namespace containing the various filling strategies."
    def median  (c,fill): return c.median()
    def constant(c,fill): return fill
    def mode    (c,fill): return c.dropna().value_counts().idxmax()

In [ ]:
#export
class FillMissing(TabularProc):
    "Fill the missing values in continuous columns."
    def __init__(self, fill_strategy=FillStrategy.median, add_col=True, fill_vals=None):
        if fill_vals is None: fill_vals = defaultdict(int)
        store_attr(self, 'fill_strategy,add_col,fill_vals')

    def setups(self, dsrc):
        self.na_dict = {n:self.fill_strategy(dsrc[n], self.fill_vals[n])
                        for n in pd.isnull(dsrc.conts).any().keys()}

    def encodes(self, to):
        missing = pd.isnull(to.conts)
        for n in missing.any().keys():
            assert n in self.na_dict, f"nan values in `{n}` but not in setup training set"
            to[n].fillna(self.na_dict[n], inplace=True)
            if self.add_col:
                to.loc[:,n+'_na'] = missing[n]
                if n+'_na' not in to.cat_names: to.cat_names.append(n+'_na')

In [ ]:
show_doc(FillMissing, title_level=3)

<h3 id="FillMissing" class="doc_header"><code>class</code> <code>FillMissing</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>FillMissing</code>(**`fill_strategy`**=*`'median'`*, **`add_col`**=*`True`*, **`fill_vals`**=*`None`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Fill the missing values in continuous columns.

In [ ]:
fill1,fill2,fill3 = (FillMissing(fill_strategy=s) 
                     for s in [FillStrategy.median, FillStrategy.constant, FillStrategy.mode])
df = pd.DataFrame({'a':[0,1,np.nan,1,2,3,4]})
df1 = df.copy(); df2 = df.copy()
tos = TabularPandas(df, fill1, cont_names='a'),TabularPandas(df1, fill2, cont_names='a'),TabularPandas(df2, fill3, cont_names='a')
test_eq(fill1.na_dict, {'a': 1.5})
test_eq(fill2.na_dict, {'a': 0})
test_eq(fill3.na_dict, {'a': 1.0})

for t in tos: test_eq(t.cat_names, ['a_na'])

for to_,v in zip(tos, [1.5, 0., 1.]):
    test_eq(to_.a.values, np.array([0, 1, v, 1, 2, 3, 4]))
    test_eq(to_.a_na.values, np.array([0, 0, 1, 0, 0, 0, 0]))

In [ ]:
dfa = pd.DataFrame({'a':[np.nan,0,np.nan]})
tos = [t.new(o) for t,o in zip(tos,(dfa,dfa.copy(),dfa.copy()))]
for t in tos: t.process()
for to_,v in zip(tos, [1.5, 0., 1.]):
    test_eq(to_.a.values, np.array([v, 0, v]))
    test_eq(to_.a_na.values, np.array([1, 0, 1]))

## TabularPandas Pipelines -

In [ ]:
procs = [Normalize, Categorify, FillMissing, noop]
df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4]})
to = TabularPandas(df, procs, cat_names='a', cont_names='b')

#Test setup and apply on df_main
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a, [1,2,3,2,2,3,1])
test_eq(to.b_na, [1,1,2,1,1,1,1])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(to.b.values, (x-m)/s)
test_eq(to.procs.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True]})

In [ ]:
#Test apply on y_names
df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to = TabularPandas(df, procs, 'a', 'b', y_names='c')

test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a, [1,2,3,2,2,3,1])
test_eq(to.b_na, [1,1,2,1,1,1,1])
test_eq(to.c, [1,0,1,0,0,1,0])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(to.b.values, (x-m)/s)
test_eq(to.procs.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True]})
test_eq(to.procs.vocab, ['a','b'])

In [ ]:
df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to = TabularPandas(df, procs, 'a', 'b', y_names='c')

test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a, [1,2,3,2,2,3,1])
test_eq(df.a.dtype,int)
test_eq(to.b_na, [1,1,2,1,1,1,1])
test_eq(to.c, [1,0,1,0,0,1,0])

In [ ]:
df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,np.nan,1,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to = TabularPandas(df, procs, cat_names='a', cont_names='b', y_names='c', splits=[[0,1,4,6], [2,3,5]])

test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a, [1,2,2,1,0,2,0])
test_eq(df.a.dtype,int)
test_eq(to.b_na, [1,2,1,1,1,1,1])
test_eq(to.c, [1,0,0,0,1,0,1])

In [ ]:
#export
class ReadTabBatch(ItemTransform):
    order = -1 #run before cuda
    def __init__(self, to): self.to = to
    # TODO: use float for cont targ
    def encodes(self, to): return tensor(to.cats).long(),tensor(to.conts).float(), tensor(to.targ)

    def decodes(self, o):
        cats,conts,targs = to_np(o)
        vals = np.concatenate([cats,conts,targs], axis=1)
        df = pd.DataFrame(vals, columns=self.to.all_col_names)
        to = self.to.new(df)
        to = self.to.procs.decode(to)
        return to

In [ ]:
#export
@typedispatch
def show_batch(x: Tabular, y, its, max_n=10, ctxs=None):
    x.show()

In [ ]:
#export
@delegates()
class TabDataLoader(TfmdDL):
    do_item = noops
    def __init__(self, dataset, bs=16, shuffle=False, after_batch=None, num_workers=0, **kwargs):
        after_batch = L(after_batch)+ReadTabBatch(dataset)
        super().__init__(dataset, bs=bs, shuffle=shuffle, after_batch=after_batch, num_workers=num_workers, **kwargs)

    def create_batch(self, b): return self.dataset.iloc[b]

TabularPandas._dl_type = TabDataLoader

## Integration example

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df_main,df_test = df.iloc[:10000].copy(),df.iloc[10000:].copy()
df_main.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]
splits = RandomSplitter()(range_of(df_main))

In [ ]:
%time to = TabularPandas(df_main, procs, cat_names, cont_names, y_names="salary", splits=splits)

CPU times: user 184 ms, sys: 16 ms, total: 200 ms
Wall time: 199 ms


In [ ]:
dbch = to.databunch()
dbch.valid_dl.show_batch()

,workclass,education,marital-status,occupation,relationship,race,age_na,fnlwgt_na,education-num_na,age,fnlwgt,education-num,salary
0,Private,Masters,Married-civ-spouse,Handlers-cleaners,Husband,White,False,False,False,63.0,167966.999571,14.0,<50k
1,Private,Assoc-acdm,Never-married,Sales,Not-in-family,White,False,False,False,26.0,94936.000008,12.0,<50k
2,Private,10th,Separated,Priv-house-serv,Not-in-family,Black,False,False,False,59.0,151976.999931,6.0,<50k
3,Private,Some-college,Never-married,Craft-repair,Unmarried,White,False,False,False,30.0,227359.001181,10.0,<50k
4,Private,Some-college,Never-married,Transport-moving,Not-in-family,White,False,False,False,20.0,231980.999018,10.0,<50k
5,Private,Some-college,Never-married,Craft-repair,Own-child,White,False,False,False,22.0,138512.998008,10.0,<50k
6,Local-gov,Assoc-voc,Never-married,Adm-clerical,Not-in-family,White,False,False,False,37.0,144321.999214,11.0,<50k
7,Private,HS-grad,Married-civ-spouse,#na#,Husband,White,False,False,False,60.0,245062.000287,9.0,>=50k
8,Private,Assoc-acdm,Never-married,Craft-repair,Not-in-family,White,False,False,True,29.0,180757.999906,10.0,<50k
9,?,Assoc-voc,Married-civ-spouse,?,Wife,White,False,False,False,41.0,211873.000594,11.0,<50k


In [ ]:
to_tst = to.new(df_test)
to_tst.process()
to_tst.all_cols.head()

,workclass,education,marital-status,occupation,relationship,race,age_na,fnlwgt_na,education-num_na,age,fnlwgt,education-num,salary
10000,5,10,3,2,1,2,1,1,1,0.471272,1.343663,1.165222,0
10001,5,12,3,15,1,4,1,1,1,-0.928063,1.256505,-0.434602,0
10002,5,2,1,9,2,5,1,1,1,1.060466,0.152101,-1.234515,0
10003,5,12,7,2,5,5,1,1,1,0.544921,-0.281682,-0.434602,0
10004,6,9,3,5,1,5,1,1,1,0.765869,1.446358,0.365310,1


## Other target types

### Multi-label categories

#### one-hot encoded label

In [ ]:
def _mock_multi_label(df):
    sal,sex,white = [],[],[]
    for row in df.itertuples():
        sal.append(row.salary == '>=50k')
        sex.append(row.sex == ' Male')
        white.append(row.race == ' White')
    df['salary'] = np.array(sal)
    df['male']   = np.array(sex)
    df['white']  = np.array(white)
    return df

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df_main,df_test = df.iloc[:10000].copy(),df.iloc[10000:].copy()
df_main = _mock_multi_label(df_main)

In [ ]:
df_main.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary,male,white
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,True,False,True
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,True,True,True
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,False,False,False
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,True,True,False
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,False,False,False


In [ ]:
#export
@EncodedMultiCategorize
def encodes(self, to:Tabular): return to
  
@EncodedMultiCategorize
def decodes(self, to:Tabular): 
    to.transform(to.y_names, lambda c: c==1)
    return to

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]
splits = RandomSplitter()(range_of(df_main))
y_names=["salary", "male", "white"]

In [ ]:
%time to = TabularPandas(df_main, procs, cat_names, cont_names, y_names=y_names, block_y=MultiCategoryBlock(encoded=True, vocab=y_names), splits=splits)

CPU times: user 172 ms, sys: 3.93 ms, total: 176 ms
Wall time: 176 ms


In [ ]:
dbch = to.databunch()
dbch.valid_dl.show_batch()

,workclass,education,marital-status,occupation,relationship,race,age_na,fnlwgt_na,education-num_na,age,fnlwgt,education-num,salary,male,white
0,Private,HS-grad,Divorced,Craft-repair,Not-in-family,White,False,False,False,59.0,110819.999452,9.0,True,True,True
1,Private,7th-8th,Never-married,Other-service,Own-child,White,False,False,False,39.0,194287.000021,4.0,False,True,True
2,Private,Assoc-voc,Never-married,Prof-specialty,Not-in-family,White,False,False,False,30.0,161690.000064,11.0,False,False,True
3,Private,Bachelors,Married-civ-spouse,Adm-clerical,Husband,White,False,False,False,38.0,156727.998594,13.0,False,True,True
4,Private,Masters,Married-civ-spouse,Exec-managerial,Husband,Black,False,False,False,38.0,103407.997765,14.0,False,True,False
5,Private,10th,Divorced,Craft-repair,Unmarried,White,False,False,False,37.0,385452.005934,6.0,False,False,True
6,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,False,False,29.0,204984.000216,9.0,False,True,True
7,Self-emp-not-inc,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,False,False,37.0,68899.003707,9.0,False,True,True
8,Private,Bachelors,Never-married,Farming-fishing,Not-in-family,White,False,False,False,31.0,356688.999826,13.0,False,True,True
9,Federal-gov,Some-college,Married-civ-spouse,Prof-specialty,Husband,White,False,False,False,45.0,155659.000831,10.0,True,True,True


#### Not one-hot encoded

In [ ]:
def _mock_multi_label(df):
    targ = []
    for row in df.itertuples():
        labels = []
        if row.salary == '>=50k': labels.append('>50k')
        if row.sex == ' Male':   labels.append('male')
        if row.race == ' White': labels.append('white')
        targ.append(' '.join(labels))
    df['target'] = np.array(targ)
    return df

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df_main,df_test = df.iloc[:10000].copy(),df.iloc[10000:].copy()
df_main = _mock_multi_label(df_main)

In [ ]:
df_main.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary,target
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k,>50k white
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k,>50k male white
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k,
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k,>50k male
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k,


In [ ]:
@MultiCategorize
def encodes(self, to:Tabular): 
    #to.transform(to.y_names, partial(_apply_cats, {n: self.vocab for n in to.y_names}, 0))
    return to
  
@MultiCategorize
def decodes(self, to:Tabular): 
    #to.transform(to.y_names, partial(_decode_cats, {n: self.vocab for n in to.y_names}))
    return to

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]
splits = RandomSplitter()(range_of(df_main))

In [ ]:
%time to = TabularPandas(df_main, procs, cat_names, cont_names, y_names="target", block_y=MultiCategoryBlock(), splits=splits)

CPU times: user 175 ms, sys: 11.9 ms, total: 187 ms
Wall time: 186 ms


In [ ]:
to.procs[2].vocab

(#24) [-,_,a,c,d,e,f,g,h,i...]

### Regression

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df_main,df_test = df.iloc[:10000].copy(),df.iloc[10000:].copy()
df_main = _mock_multi_label(df_main)

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]
splits = RandomSplitter()(range_of(df_main))

In [ ]:
%time to = TabularPandas(df_main, procs, cat_names, cont_names, y_names='age', block_y=TransformBlock(), splits=splits)

CPU times: user 166 ms, sys: 4.08 ms, total: 170 ms
Wall time: 168 ms


In [ ]:
to.procs[-1].means

fnlwgt           192101.717625
education-num        10.074750
dtype: float64

In [ ]:
dbch = to.databunch()
dbch.valid_dl.show_batch()

,workclass,education,marital-status,occupation,relationship,race,fnlwgt_na,education-num_na,fnlwgt,education-num,age
0,Private,Masters,Divorced,Sales,Not-in-family,White,False,False,182628.999640,14.0,44.0
1,Private,9th,Never-married,Machine-op-inspct,Unmarried,Black,False,False,122472.999677,5.0,43.0
2,Private,HS-grad,Married-AF-spouse,Other-service,Wife,White,False,False,30565.001977,9.0,35.0
3,Private,10th,Married-civ-spouse,#na#,Husband,Black,False,False,180446.000134,6.0,48.0
4,Private,HS-grad,Never-married,Adm-clerical,Not-in-family,White,False,False,314798.003810,9.0,26.0
5,Private,Some-college,Married-civ-spouse,Other-service,Husband,White,False,False,121124.000575,10.0,48.0
6,Federal-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,False,False,155593.999236,13.0,53.0
7,Private,HS-grad,Never-married,Other-service,Own-child,Black,False,False,266635.000142,9.0,19.0
8,Local-gov,Some-college,Married-civ-spouse,Protective-serv,Husband,White,False,False,267858.999576,10.0,33.0
9,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,False,177181.000675,9.0,37.0


## Not being used now - for multi-modal

In [ ]:
class TensorTabular(Tuple):
    def get_ctxs(self, max_n=10, **kwargs):
        n_samples = min(self[0].shape[0], max_n)
        df = pd.DataFrame(index = range(n_samples))
        return [df.iloc[i] for i in range(n_samples)]

    def display(self, ctxs): display_df(pd.DataFrame(ctxs))

class TabularLine(pd.Series):
    "A line of a dataframe that knows how to show itself"
    def show(self, ctx=None, **kwargs): return self if ctx is None else ctx.append(self)

class ReadTabLine(ItemTransform):
    def __init__(self, proc): self.proc = proc

    def encodes(self, row):
        cats,conts = (o.map(row.__getitem__) for o in (self.proc.cat_names,self.proc.cont_names))
        return TensorTabular(tensor(cats).long(),tensor(conts).float())

    def decodes(self, o):
        to = TabularPandas(o, self.proc.cat_names, self.proc.cont_names, self.proc.y_names)
        to = self.proc.decode(to)
        return TabularLine(pd.Series({c: v for v,c in zip(to.items[0]+to.items[1], self.proc.cat_names+self.proc.cont_names)}))

class ReadTabTarget(ItemTransform):
    def __init__(self, proc): self.proc = proc
    def encodes(self, row): return row[self.proc.y_names].astype(np.int64)
    def decodes(self, o): return Category(self.proc.classes[self.proc.y_names][o])

In [ ]:
# tds = TfmdDS(to.items, tfms=[[ReadTabLine(proc)], ReadTabTarget(proc)])
# enc = tds[1]
# test_eq(enc[0][0], tensor([2,1]))
# test_close(enc[0][1], tensor([-0.628828]))
# test_eq(enc[1], 1)

# dec = tds.decode(enc)
# assert isinstance(dec[0], TabularLine)
# test_close(dec[0], pd.Series({'a': 1, 'b_na': False, 'b': 1}))
# test_eq(dec[1], 'a')

# test_stdout(lambda: print(show_at(tds, 1)), """a               1
# b_na        False
# b               1
# category        a
# dtype: object""")

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_test.ipynb.
Converted 01_core.foundation.ipynb.
Converted 01a_core.utils.ipynb.
Converted 01b_core.dispatch.ipynb.
Converted 01c_core.transform.ipynb.
Converted 02_core.script.ipynb.
Converted 03_torch_core.ipynb.
Converted 03a_layers.ipynb.
Converted 04_data.load.ipynb.
Converted 05_data.core.ipynb.
Converted 06_data.transforms.ipynb.
Converted 07_data.block.ipynb.
Converted 08_vision.core.ipynb.
Converted 09_vision.augment.ipynb.
Converted 09a_vision.data.ipynb.
Converted 09b_vision.utils.ipynb.
Converted 10_tutorial.pets.ipynb.
Converted 11_vision.models.xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 13a_metrics.ipynb.
Converted 14_callback.schedule.ipynb.
Converted 14a_callback.data.ipynb.
Converted 15_callback.hook.ipynb.
Converted 15a_vision.models.unet.ipynb.
Converted 16_callback.progress.ipynb.
Converted 17_callback.tracker.ipynb.
Converted 18_callback.fp16.ipynb.
Converted 19_callback.mixup.ipynb.
Converted 20_interpret.ipynb.
